Regresión Logística: Explicación Detallada
1. Introducción
La regresión logística es un modelo estadístico utilizado para la clasificación binaria y multiclase. A diferencia de la regresión lineal, donde la variable dependiente es continua, la regresión logística predice probabilidades y se usa para modelar una variable dependiente categórica.
Por ejemplo, en clasificación binaria (0 o 1), el modelo estima la probabilidad de que una observación pertenezca a una de las dos clases. Si la probabilidad es mayor a un umbral (generalmente 0.5), la predicción será 1, en caso contrario, será 0.
2. Modelo Matemático
2.1. Función Sigmoide
La función principal en la regresión logística es la sigmoide o logística, definida como:
σ(z)=11+e−z\sigma(z) = \frac{1}{1 + e^{-z}}σ(z)=1+e−z1
Donde:
•	z=β0+β1x1+β2x2+...+βnxnz = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + ... + \beta_n x_nz=β0+β1x1+β2x2+...+βnxn es la combinación lineal de los predictores.
•	σ(z)\sigma(z)σ(z) transforma cualquier valor real en un rango entre (0,1), interpretado como una probabilidad.


In [ ]:
from sklearn.datasets import make_moons, make_regression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.ensemble import RandomForestRegressor, VotingRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score

### CLASIFICADOR ###
class VotingEnsembleClassifier:
    def __init__(self, voting_type='soft'):
        """Inicializa el ensemble de clasificadores."""
        self.voting_clf = VotingClassifier(
            estimators=[
                ('lr', LogisticRegression(random_state=42)),
                ('rf', RandomForestClassifier(random_state=42)),
                ('svc', SVC(probability=True, random_state=42)),
                ('knn', KNeighborsClassifier(n_neighbors=5)),
                ('nb', GaussianNB()),
                ('gb', GradientBoostingClassifier(random_state=42)),
                ('mlp', MLPClassifier(hidden_layer_sizes=(100,), random_state=42)),
                ('et', ExtraTreesClassifier(random_state=42))
            ],
            voting=voting_type
        )

    def train(self, X_train, y_train):
        """Entrena el modelo con los datos de entrenamiento."""
        self.voting_clf.fit(X_train, y_train)

    def predict(self, X_test):
        """Realiza predicciones en los datos de prueba."""
        return self.voting_clf.predict(X_test)

    def evaluate(self, X_test, y_test):
        """Calcula la precisión del modelo en los datos de prueba."""
        y_pred = self.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Precisión del VotingClassifier: {accuracy:.4f}")
        return accuracy

### REGRESOR ###
class VotingEnsembleRegressor:
    def __init__(self):
        """Inicializa el ensemble de regresores."""
        self.voting_reg = VotingRegressor(
            estimators=[
                ('lr', LinearRegression()),
                ('rf', RandomForestRegressor(random_state=42)),
                ('svr', SVR()),
                ('knn', KNeighborsRegressor(n_neighbors=5)),
                ('gb', GradientBoostingRegressor(random_state=42)),
                ('mlp', MLPRegressor(hidden_layer_sizes=(100,), random_state=42)),
                ('et', ExtraTreesRegressor(random_state=42))
            ]
        )

    def train(self, X_train, y_train):
        """Entrena el modelo con los datos de entrenamiento."""
        self.voting_reg.fit(X_train, y_train)

    def predict(self, X_test):
        """Realiza predicciones en los datos de prueba."""
        return self.voting_reg.predict(X_test)

    def evaluate(self, X_test, y_test):
        """Calcula métricas de evaluación para regresión."""
        y_pred = self.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        print(f"Error cuadrático medio (MSE): {mse:.4f}")
        print(f"Coeficiente de determinación (R²): {r2:.4f}")
        return mse, r2

# ====== CLASIFICACIÓN ======
print("\n---- CLASIFICACIÓN ----")
X_class, y_class = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_class, y_class, random_state=42)

ensemble_classifier = VotingEnsembleClassifier(voting_type='soft')
ensemble_classifier.train(X_train_c, y_train_c)
ensemble_classifier.evaluate(X_test_c, y_test_c)

# ====== REGRESIÓN ======
print("\n---- REGRESIÓN ----")
X_reg, y_reg = make_regression(n_samples=500, n_features=5, noise=10, random_state=42)
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_reg, y_reg, random_state=42)

ensemble_regressor = VotingEnsembleRegressor()
ensemble_regressor.train(X_train_r, y_train_r)
ensemble_regressor.evaluate(X_test_r, y_test_r)

In [ ]:
import shap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# === 1. DEFINIR VARIABLES ===
# Separa las características y la variable objetivo
X = df_stan.drop(['Precio_euros'], axis=1)
y = df_stan['Precio_euros']

# Divide los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# === 2. ENTRENAR MODELO RANDOM FOREST ===
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predicciones y evaluación
y_pred = rf_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"🔹 Error cuadrático medio (MSE): {mse:.4f}")
print(f"🔹 Coeficiente de determinación (R²): {r2:.4f}")

# === 3. IMPORTANCIA DE CARACTERÍSTICAS CON SHAP ===
class FeatureImportanceSHAP:
    def __init__(self, model, X_train):
        """
        Clase para calcular la importancia de características usando SHAP.
        
        :param model: Modelo de Machine Learning ya entrenado.
        :param X_train: Datos de entrenamiento (sin la variable objetivo).
        """
        self.model = model
        self.X_train = X_train
        self.explainer = shap.Explainer(model, X_train)
        self.shap_values = self.explainer(X_train)

    def get_importance(self):
        """Devuelve un DataFrame con la importancia media de cada característica."""
        mean_abs_shap = np.abs(self.shap_values.values).mean(axis=0)
        feature_names = self.X_train.columns if isinstance(self.X_train, pd.DataFrame) else [f"Feature {i}" for i in range(self.X_train.shape[1])]

        importance_df = pd.DataFrame({'Feature': feature_names, 'SHAP Importance': mean_abs_shap})
        return importance_df.sort_values(by='SHAP Importance', ascending=False)

    def plot_summary(self):
        """Genera el gráfico de importancia de características con SHAP."""
        shap.summary_plot(self.shap_values, self.X_train)

    def plot_dependence(self, feature_name):
        """
        Genera un gráfico de dependencia para una característica específica.

        :param feature_name: Nombre de la característica a analizar.
        """
        if isinstance(self.X_train, pd.DataFrame):
            shap.dependence_plot(feature_name, self.shap_values.values, self.X_train)
        else:
            print("Error: Para graficar dependencia, X_train debe ser un DataFrame con nombres de columnas.")

# === 4. ANÁLISIS CON SHAP ===
shap_analysis = FeatureImportanceSHAP(rf_model, X_train)

# Mostrar la importancia de cada característica
print("\n🔹 Importancia de Características según SHAP:")
print(shap_analysis.get_importance())

# Generar gráfico SHAP de importancia general
shap_analysis.plot_summary()

# Graficar la relación de una característica específica (ejemplo: "RAM_GB")
shap_analysis.plot_dependence(feature_name="RAM")